In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'
df = pd.read_csv('../day5/final_부동산.csv')

In [55]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
df_부동산 = df[df['업종(대분류)'].str.contains('부동산업', na=False)]

In [57]:
df_부동산['기준년'] = df_부동산['기준년월']
df_부동산['기준월'] = df_부동산['기준년월']

In [58]:
# 총자산 = 유동자산 + 비유동자산
df_부동산['총자산'] = df_부동산['유동자산'] + df_부동산['비유동자산']

# 총부채 = 유동부채 + 비유동부채
df_부동산['총부채'] = df_부동산['유동부채'] + df_부동산['비유동부채']

# 부채비율 = 총부채 / 총자산
df_부동산['부채비율'] = df_부동산['총부채'] / (df_부동산['총자산'] + 1e-6)  # 0으로 나누는 것 방지

In [59]:
# 자기자본 = 총자산 - 총부채
df_부동산['자기자본'] = df_부동산['총자산'] - df_부동산['총부채']

# 자기자본비율 = 자기자본 / 총자산
df_부동산['자기자본비율'] = df_부동산['자기자본'] / (df_부동산['총자산'] + 1e-6)

# 총자산회전율 = 매출액 / 총자산
df_부동산['총자산회전율'] = df_부동산['매출액'] / (df_부동산['총자산'] + 1e-6)

# 매출총이익률 = 매출총이익 / 매출액
df_부동산['매출총이익률'] = df_부동산['매출총이익'] / (df_부동산['매출액'] + 1e-6)


In [60]:
for col in df_부동산.columns:
    if '30' in col:
        print(col)

기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)
기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함)
기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함)
기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함)
기업신용공여30일이상연체과목수(일보)(해제포함)
기업신용공여30일이상연체과목수(일보)(미해제)
기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함)
기업신용공여30일이상연체과목수(일보)(이자연체)(미해제)
기업신용공여30일이상연체기관수(일보)(해제포함)
기업신용공여30일이상연체기관수(일보)(미해제)
기업신용공여30일이상연체기관수(일보)(연체)(해제포함)
기업신용공여30일이상연체기관수(일보)(이자연체)(미해제)
기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)
기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함)
기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함)
기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함)


In [61]:
# 연체 경험이 있는지 여부 (3년 이내 30일 이상 연체)
df_부동산['연체경험여부'] = (
    (df_부동산['기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)'] > 0)
).astype(int)

# 최근 연체 발생 후 경과일수 (음수는 없음으로 처리)
df_부동산['연체발생_경과일수'] = df_부동산[
    '신용도판단정보공공정보최근발생일자로부터경과일수(CIS)(해제,삭제)'
].clip(lower=0)

# 공공정보 유지건수 존재 여부
df_부동산['공공정보_유지여부'] = (
    df_부동산['공공정보(국세,지방세,관세체납)건수(CIS)(미해제)'] > 0
).astype(int)


In [62]:
df_부동산['총연체대출과목수'] = (
    df_부동산['기업신용공여연체과목수(일보)(미해제)']+
    df_부동산['기업신용공여연체과목수(일보)(3개월내유지)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(6개월내유지)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(1년내유지)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(3년내유지)(해제포함)']
)

df_부동산['총신규연체과목수'] = (
    df_부동산['기업신용공여연체과목수(일보)(3개월내발생)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(6개월내발생)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(1년내발생)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(3년내발생)(해제포함)']
)

In [63]:
df_부동산['총장기연체과목수'] = (
    df_부동산['기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(6개월내유지)(연체일수30일이상)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(1년내유지)(연체일수30일이상)(해제포함)']+
    df_부동산['기업신용공여연체과목수(일보)(3년내유지)(연체일수30일이상)(해제포함)']
)

df_부동산['이자원금_30일초과연체과목수'] = (
    df_부동산['기업신용공여30일이상연체과목수(일보)(이자연체)(해제포함)']+
    df_부동산['기업신용공여30일이상연체과목수(일보)(이자연체)(미해제)']
)

In [73]:
rename_map = {
    '기업신용공여연체기관수(일보)(3년내유지)(연체일수30일이상)(해제포함)': '연체기관수_3년',
    '기업신용공여연체기관수(일보)(1년내유지)(연체일수30일이상)(해제포함)': '연체기관수_1년',
    '기업신용공여연체기관수(일보)(6개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_6개월',
    '기업신용공여연체기관수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체기관수_3개월',
    '기업신용공여연체과목수(일보)(3개월내발생)(해제포함)': '연체과목수_3개월발생',
    '기업신용공여연체과목수(일보)(3개월내유지)(연체일수30일이상)(해제포함)': '연체과목수_3개월유지',
    '기업신용공여30일이상연체기관수(일보)(해제포함)': '연체기관수_전체',
    '기업신용공여연체최장연체일수(일보)(3개월내발생)(해제포함)': '최장연체일수_3개월',
    '기업신용공여연체최장연체일수(일보)(6개월내발생)(해제포함)': '최장연체일수_6개월',
    '기업신용공여연체최장연체일수(일보)(1년내발생)(해제포함)': '최장연체일수_1년',
    '기업신용공여연체최장연체일수(일보)(3년내발생)(해제포함)': '최장연체일수_3년',
    '연체경험여부': '연체경험',
}


In [74]:
df_부동산.rename(columns=rename_map, inplace=True)


In [75]:
selected_features = [
    # 상관계수 상위 피처
    '연체기관수_3년', '연체기관수_1년', '연체기관수_6개월', '연체기관수_3개월',
    '연체과목수_3개월발생', '연체과목수_3개월유지', '연체기관수_전체',
    '최장연체일수_3개월', '최장연체일수_6개월', '최장연체일수_1년', '최장연체일수_3년',
    '연체경험',

    # 재무 지표
    '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계',
    '매출액', '매출총이익', '영업손익', '당기순이익', '영업활동현금흐름',
    '재무비율_부채비율', '재무비율_유동비율', '재무비율_자기자본비율',
    '재무비율_영업이익율', '재무비율_당기순이익율', '재무비율_자기자본이익률(ROE)',
    'EBITDA마진율', '영업이익증가율', '당기순이익증가율', 'EBITDA증가율',

    # 기타
    '설립일자', '상장일자', '사업장소유여부', '소유건축물건수', '소유건축물권리침해여부',
    '기업신용평가등급(구간화)', '공공정보_유지여부',

    # 타겟
    '모형개발용Performance(향후1년내부도여부)'
]


In [76]:
df_부동산['모형개발용Performance(향후1년내부도여부)'].value_counts(normalize=True)


모형개발용Performance(향후1년내부도여부)
0    0.970596
1    0.029404
Name: proportion, dtype: float64

In [77]:
df = df_부동산[selected_features]

In [78]:
df.groupby('모형개발용Performance(향후1년내부도여부)')[list(rename_map.values())].mean().T.sort_values(by=1, ascending=False)


모형개발용Performance(향후1년내부도여부),0,1
최장연체일수_3년,3.491764,16.165926
최장연체일수_1년,0.973778,6.139630
최장연체일수_6개월,0.538486,3.394444
최장연체일수_3개월,0.275111,2.243704
연체기관수_전체,0.062688,0.219630
연체기관수_3년,0.048505,0.198519
연체과목수_3개월발생,0.026772,0.186667
연체기관수_1년,0.016988,0.130000
연체기관수_6개월,0.010424,0.110741
연체기관수_3개월,0.006463,0.084444


In [79]:
df_부동산[selected_features]

,연체기관수_3년,연체기관수_1년,연체기관수_6개월,연체기관수_3개월,연체과목수_3개월발생,연체과목수_3개월유지,연체기관수_전체,최장연체일수_3개월,최장연체일수_6개월,최장연체일수_1년,...,당기순이익증가율,EBITDA증가율,설립일자,상장일자,사업장소유여부,소유건축물건수,소유건축물권리침해여부,기업신용평가등급(구간화),공공정보_유지여부,모형개발용Performance(향후1년내부도여부)
0,0,0,0,0,0,0,0,0,0,0,...,39.51,139.55,20020830,NaN,0,1595,1,4,0,0
1,0,0,0,0,0,0,0,0,0,0,...,-17.75,-856.00,19850611,NaN,0,99,0,5,0,0
2,0,0,0,0,0,0,0,0,0,0,...,16.57,-59.07,20070115,NaN,1,47,1,2,0,0
3,0,0,0,0,0,0,0,0,0,0,...,-11.57,0.00,19990201,NaN,0,44,0,3,0,0
4,0,0,0,0,0,0,0,0,0,0,...,420.22,3.00,19880812,NaN,1,144,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91819,0,0,0,0,0,0,0,0,0,0,...,-161.60,328.00,20190619,NaN,0,0,0,3,0,0
91820,0,0,0,0,0,0,0,0,0,0,...,-17.00,NaN,20170806,NaN,0,0,0,3,0,0
91821,1,0,0,0,0,0,1,0,0,0,...,-98.59,21.08,20090928,NaN,0,0,0,2,0,0
91822,0,0,0,0,0,0,0,0,0,0,...,-107.90,9.00,20180225,NaN,0,0,0,6,0,0


In [80]:
df = df_부동산[selected_features]

In [81]:
print(df.describe())

           연체기관수_3년      연체기관수_1년     연체기관수_6개월     연체기관수_3개월   연체과목수_3개월발생  \
count  91824.000000  91824.000000  91824.000000  91824.000000  91824.000000   
mean       0.052916      0.020311      0.013373      0.008756      0.031473   
std        0.340754      0.211954      0.197993      0.176144      0.358800   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        8.000000      7.000000      9.000000      9.000000     20.000000   

        연체과목수_3개월유지      연체기관수_전체    최장연체일수_3개월    최장연체일수_6개월     최장연체일수_1년  \
count  91824.000000  91824.000000  91824.000000  91824.000000  91824.000000   
mean       0.007612      0.067303      0.332996      0.622463      1.125675   
std        0.169487      0.397767      3.241048    

In [82]:
df.describe(include='all')


,연체기관수_3년,연체기관수_1년,연체기관수_6개월,연체기관수_3개월,연체과목수_3개월발생,연체과목수_3개월유지,연체기관수_전체,최장연체일수_3개월,최장연체일수_6개월,최장연체일수_1년,...,당기순이익증가율,EBITDA증가율,설립일자,상장일자,사업장소유여부,소유건축물건수,소유건축물권리침해여부,기업신용평가등급(구간화),공공정보_유지여부,모형개발용Performance(향후1년내부도여부)
count,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,...,9.182400e+04,8.784600e+04,9.182400e+04,4.950000e+02,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000,91824.000000
mean,0.052916,0.020311,0.013373,0.008756,0.031473,0.007612,0.067303,0.332996,0.622463,1.125675,...,1.024354e+02,-1.431989e+03,2.010128e+07,1.999666e+07,0.071463,39.990656,0.131382,5.270735,0.013384,0.029404
std,0.340754,0.211954,0.197993,0.176144,0.358800,0.169487,0.397767,3.241048,5.164625,9.084944,...,2.407384e+04,9.618671e+04,9.458638e+04,1.392667e+05,0.257598,145.819942,0.337819,1.887303,0.114914,0.168937
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-8.636362e+05,-7.325767e+06,1.945091e+07,1.963102e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.277375e+02,-6.300000e+01,2.007053e+07,1.993093e+07,0.000000,1.000000,0.000000,4.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.049000e+01,0.000000e+00,2.013092e+07,1.995061e+07,0.000000,5.000000,0.000000,5.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.973000e+01,5.000000e+00,2.016072e+07,2.013111e+07,0.000000,26.000000,0.000000,7.000000,0.000000,0.000000
max,8.000000,7.000000,9.000000,9.000000,20.000000,26.000000,12.000000,89.000000,161.000000,341.000000,...,4.040773e+06,4.307240e+05,2.021083e+07,2.022081e+07,1.000000,3206.000000,1.000000,10.000000,1.000000,1.000000


In [83]:
df.isnull().sum().sort_values(ascending=False)


상장일자                           91329
재무비율_당기순이익율                    23616
EBITDA증가율                       3978
연체기관수_3년                           0
영업이익증가율                            0
재무비율_부채비율                          0
재무비율_유동비율                          0
재무비율_자기자본비율                        0
재무비율_영업이익율                         0
재무비율_자기자본이익률(ROE)                  0
EBITDA마진율                          0
당기순이익증가율                           0
당기순이익                              0
설립일자                               0
사업장소유여부                            0
소유건축물건수                            0
소유건축물권리침해여부                        0
기업신용평가등급(구간화)                      0
공공정보_유지여부                          0
영업활동현금흐름                           0
영업손익                               0
연체기관수_1년                           0
매출총이익                              0
연체기관수_6개월                          0
연체기관수_3개월                          0
연체과목수_3개월발생                        0
연체과목수_3개월유지                        0
연

In [84]:
import pandas as pd

# 상관관계 계산을 위한 복사본 생성
df_corr = df.copy()

# 날짜 컬럼 숫자로 변환 (결측치는 제외)
df_corr['상장일자'] = pd.to_numeric(df_corr['상장일자'], errors='coerce')

# 상관관계 계산 (결측치 자동 제거됨)
corr_value = df_corr[['상장일자', '모형개발용Performance(향후1년내부도여부)']].corr().iloc[0, 1]
print(f"📌 상장일자와 타겟 간의 상관계수: {corr_value:.4f}")


📌 상장일자와 타겟 간의 상관계수: 0.1357


In [85]:
df.drop(columns=['상장일자'], inplace=True)


In [86]:
correlation = df['재무비율_당기순이익율'].corr(df['모형개발용Performance(향후1년내부도여부)'])
print(f'📌 재무비율_당기순이익율과 타겟 간의 상관계수: {correlation:.4f}')

📌 재무비율_당기순이익율과 타겟 간의 상관계수: 0.0005


In [87]:
df.drop(columns=["재무비율_당기순이익율"], inplace=True)


In [88]:
df = df[df["EBITDA증가율"].notna()]


In [89]:
print("✅ 제거 후 샘플 수:", len(df))


✅ 제거 후 샘플 수: 87846


In [90]:
null_counts = df.isnull().sum()
null_counts[null_counts > 0]


Series([], dtype: int64)

In [91]:
# 음수면 안 되는 컬럼 목록
must_be_positive = [
    '부채총계', '유동자산', '비유동자산', '자산총계',
    '유동부채', '비유동부채',
    '연체기관수_3년', '연체기관수_1년', '연체기관수_6개월',
    '연체기관수_3개월', '연체과목수_3개월발생', '연체과목수_3개월유지',
    '연체기관수_전체', '최장연체일수_3개월', '최장연체일수_6개월', '최장연체일수_1년',
    '설립일자', '소유건축물건수'
]

# 이상치 제거 (음수값 있는 행 제거)
for col in must_be_positive:
    df_temp = df[df[col] >= 0]


In [92]:
print(f"📊 현재 샘플 수: {df_temp.shape[0]}")
print(f"📊 현재 컬럼 수: {df_temp.shape[1]}")


📊 현재 샘플 수: 87846
📊 현재 컬럼 수: 39


In [93]:
df['모형개발용Performance(향후1년내부도여부)'].value_counts(normalize=True)

모형개발용Performance(향후1년내부도여부)
0    0.97079
1    0.02921
Name: proportion, dtype: float64

In [94]:
target_col = '모형개발용Performance(향후1년내부도여부)'

from sklearn.model_selection import train_test_split

# 특성과 타겟 분리
X = df.drop(columns=[target_col])
y = df[target_col]


In [95]:
from imblearn.over_sampling import SMOTE

# SMOTE 객체 생성
smote = SMOTE(random_state=42)

# 오버샘플링 적용
X_resampled, y_resampled = smote.fit_resample(X, y)

# 결과 확인
import pandas as pd
print("✅ SMOTE 적용 결과 클래스 분포:")
print(pd.Series(y_resampled).value_counts(normalize=False))


✅ SMOTE 적용 결과 클래스 분포:
모형개발용Performance(향후1년내부도여부)
0    85280
1    85280
Name: count, dtype: int64


In [96]:
from sklearn.model_selection import train_test_split

# 학습/테스트 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled,
    test_size=0.2,
    random_state=42,
    stratify=y_resampled
)

print(f"✅ 학습 데이터 수: {len(X_train)}")
print(f"✅ 테스트 데이터 수: {len(X_test)}")


✅ 학습 데이터 수: 136448
✅ 테스트 데이터 수: 34112


In [97]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. 기본 학습기 구성
base_learners = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=42))
]

# 2. 스태킹 모델 정의
stack_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

# 3. 학습
stack_model.fit(X_train, y_train)

# 4. 예측
y_pred = stack_model.predict(X_test)

# 5. 평가 결과 출력
print("✅ Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=4))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:10:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[17:11:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:11:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:11:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:11:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:11:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



✅ Confusion Matrix:
[[17019    37]
 [   41 17015]]

✅ Classification Report:
              precision    recall  f1-score   support

           0     0.9976    0.9978    0.9977     17056
           1     0.9978    0.9976    0.9977     17056

    accuracy                         0.9977     34112
   macro avg     0.9977    0.9977    0.9977     34112
weighted avg     0.9977    0.9977    0.9977     34112



In [98]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# 하이퍼파라미터 그리드
param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [5, 10, 20],
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1, 0.2]
}

# 스태킹 모델 구성
stack_model = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

# RandomizedSearchCV 실행
search = RandomizedSearchCV(
    estimator=stack_model,
    param_distributions=param_grid,
    n_iter=10,
    cv=3,
    scoring='recall',  # 🎯 Recall 중심
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# 학습
search.fit(X_train, y_train)

# 결과 확인
print("📌 Best Params:", search.best_params_)
print("📌 Best Recall:", search.best_score_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[17:12:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:12:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" }

[CV] END rf__max_depth=5, rf__n_estimators=100, xgb__learning_rate=0.01, xgb__max_depth=7, xgb__n_estimators=100; total time= 1.3min
[CV] END rf__max_depth=5, rf__n_estimators=100, xgb__learning_rate=0.1, xgb__max_depth=7, xgb__n_estimators=100; total time= 1.3min


[17:13:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



[CV] END rf__max_depth=5, rf__n_estimators=100, xgb__learning_rate=0.1, xgb__max_depth=7, xgb__n_estimators=100; total time= 1.3min


...

[CV] END rf__max_depth=5, rf__n_estimators=100, xgb__learning_rate=0.1, xgb__max_depth=7, xgb__n_estimators=200; total time= 2.0min


[17:21:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:21:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:21:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:21:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



[CV] END rf__max_depth=5, rf__n_estimators=100, xgb__learning_rate=0.1, xgb__max_depth=7, xgb__n_estimators=200; total time= 2.3min
[CV] END rf__max_depth=5, rf__n_estimators=100, xgb__learning_rate=0.1, xgb__max_depth=7, xgb__n_estimators=200; total time= 2.0min
[CV] END rf__max_depth=20, rf__n_estimators=200, xgb__learning_rate=0.01, xgb__max_depth=7, xgb__n_estimators=200; total time= 7.3min
[CV] END rf__max_depth=20, rf__n_estimators=200, xgb__learning_rate=0.01, xgb__max_depth=7, xgb__n_estimators=200; total time= 7.4min
[CV] END rf__max_depth=20, rf__n_estimators=200, xgb__learning_rate=0.01, xgb__max_depth=7, xgb__n_estimators=200; total time= 7.5min


/opt/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:26:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[17:27:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:27:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:27:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:27:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[17:27:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



📌 Best Params: {'xgb__n_estimators': 200, 'xgb__max_depth': 7, 'xgb__learning_rate': 0.01, 'rf__n_estimators': 200, 'rf__max_depth': 20}
📌 Best Recall: 0.9927591375298283


In [99]:
import joblib

# best 모델 저장
joblib.dump(search.best_estimator_, 'stacking_model.pkl')
print("✅ 모델 저장 완료: stacking_model.pkl")


✅ 모델 저장 완료: stacking_model.pkl


In [100]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 최종 하이퍼파라미터
xgb = XGBClassifier(n_estimators=200, max_depth=7, learning_rate=0.01, random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)

# 스태킹 구성
stacking_model = StackingClassifier(
    estimators=[('xgb', xgb), ('rf', rf)],
    final_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
    passthrough=True
)

# 전체 파이프라인
pipeline = Pipeline([
    ("scaler", StandardScaler()),  # 수치형 데이터 스케일링
    ("model", stacking_model)
])


In [101]:
# train_test_split (예시)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 학습
pipeline.fit(X_train, y_train)

# 저장
import pickle
with open("final_risk_model.pkl", "wb") as f:
    pickle.dump(pipeline, f)


In [110]:
X.columns

Index(['연체기관수_3년', '연체기관수_1년', '연체기관수_6개월', '연체기관수_3개월', '연체과목수_3개월발생',
       '연체과목수_3개월유지', '연체기관수_전체', '최장연체일수_3개월', '최장연체일수_6개월', '최장연체일수_1년',
       '최장연체일수_3년', '연체경험', '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계',
       '매출액', '매출총이익', '영업손익', '당기순이익', '영업활동현금흐름', '재무비율_부채비율', '재무비율_유동비율',
       '재무비율_자기자본비율', '재무비율_영업이익율', '재무비율_자기자본이익률(ROE)', 'EBITDA마진율',
       '영업이익증가율', '당기순이익증가율', 'EBITDA증가율', '설립일자', '사업장소유여부', '소유건축물건수',
       '소유건축물권리침해여부', '기업신용평가등급(구간화)', '공공정보_유지여부'],
      dtype='object')

In [102]:
import pandas as pd
pd.Series(X.columns).to_csv("model_input_features.csv", index=False)

In [103]:
model_input_features = pd.Series(X.columns)

In [104]:
# 라벨이 1인 인덱스 찾기
positive_idx = y_test[y_test == 1].index

# 해당 인덱스의 X_test에서 샘플 하나 추출
sample_input = X_test.loc[positive_idx[0]]  # 첫 번째 라벨 1 샘플

In [105]:
# 모델 입력은 2D 형태로 맞춰야 하므로 DataFrame으로 변환
sample_input_df = pd.DataFrame([sample_input])


In [106]:
import pickle

# 모델 로딩
with open("final_risk_model.pkl", "rb") as f:
    model = pickle.load(f)

# 예측
pred = model.predict(sample_input_df)[0]
proba = model.predict_proba(sample_input_df)[0]

print(f"예측 결과: {'부도 위험 있음' if pred == 1 else '정상'}")
print(f"확률 (정상 / 부도위험): {proba}")

예측 결과: 부도 위험 있음
확률 (정상 / 부도위험): [0.33 0.67]


In [107]:
import pandas as pd

# 라벨이 1인 샘플 인덱스 찾기
positive_idx = y_test[y_test == 1].index

# 해당 인덱스의 샘플 추출
sample_input = X_test.loc[positive_idx[10]]

# DataFrame 형식으로 변환
sample_input_df = pd.DataFrame([sample_input])

# CSV로 저장
sample_input_df.to_csv("sample_input_label1.csv", index=False, encoding="utf-8-sig")

print("✅ sample_input_label1.csv 파일 저장 완료")


✅ sample_input_label1.csv 파일 저장 완료


In [108]:
import pandas as pd

# 라벨이 1인 인덱스 찾기
positive_indices = y_test[y_test == 1].index

# 해당 인덱스의 X값 추출
X_test_positive = X_test.loc[positive_indices]

# CSV로 저장
X_test_positive.to_csv("X_test_label1.csv", index=False, encoding="utf-8-sig")

print(f"✅ 라벨이 1인 테스트 데이터 {len(X_test_positive)}건을 X_test_label1.csv로 저장 완료")


✅ 라벨이 1인 테스트 데이터 513건을 X_test_label1.csv로 저장 완료
